In [3]:
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import matplotlib as mat
import os
import datetime
import time
from tqdm import tqdm

# mat.rcParams['font.family'] = 'SimHei'
# mat.rcParams['font.sans-serif'] = 'SimHei'

#忽略警告
import warnings
warnings.filterwarnings('ignore')

ValueError: numpy.dtype size changed, may indicate binary incompatibility. Expected 96 from C header, got 88 from PyObject

# Data Reading

In [2]:
#date
file_list = [
     '0706', '0707', '0708', '0709','0710','0711','0712', '0714', '0715',
    '0718', '0719', '0720', '0722', '0724', '0725', '0726', '0727',
    '0728', '0730', '0731', '0801', '0802', '0803','0804','0805','0806'
,'0807']

df_list = []

for i in tqdm(file_list,):
    df = pd.read_json(os.getcwd()+"\\wifi_track_Data\\dacang\\origin_data\\prob2022%s.json" %(i),lines=True)
    
    df_list.append(df)

df_read = pd.concat(df_list)
df_read

100%|██████████| 27/27 [00:30<00:00,  1.12s/it]


,_id,m,a,r,t
0,{'$oid': '60e32f77f90a6c7ac960a79b'},"[1, 128, 194, 0, 0, 0]",121,169,1.625501e+09
1,{'$oid': '60e32f77f90a6c7ac960a79c'},"[80, 2, 145, 187, 143, 221]",121,159,1.625501e+09
2,{'$oid': '60e32f77f90a6c7ac960a79d'},"[176, 89, 71, 72, 233, 219]",121,160,1.625501e+09
3,{'$oid': '60e32f77f90a6c7ac960a79e'},"[160, 65, 71, 44, 196, 200]",121,163,1.625501e+09
4,{'$oid': '60e32f77f90a6c7ac960a79f'},"[51, 51, 255, 86, 69, 185]",121,159,1.625501e+09
...,...,...,...,...,...
174556,{'$oid': '62efe057865837f6d4fc457e'},"[88, 102, 186, 101, 140, 144]",66,176,1.659887e+09
174557,{'$oid': '62efe057865837f6d4fc457f'},"[88, 102, 186, 101, 140, 144]",66,177,1.659887e+09
174558,{'$oid': '62efe057865837f6d4fc4580'},"[1, 0, 94, 127, 255, 250]",66,161,1.659887e+09
174559,{'$oid': '62efe057865837f6d4fc4581'},"[1, 0, 94, 127, 255, 250]",66,162,1.659888e+09


In [2]:
# convert time format
df = df_read.copy()

def Convert2Time(t):
    newT = time.localtime(t)
    timeStyle = time.strftime("%Y-%m-%d %H:%M:%S",newT)
    return timeStyle

df['t'] = df['t'].apply(Convert2Time)

#change 2021 -> 2022 (little data bug)
df['t'] = df['t'].apply(lambda t : t.replace('2021','2022'))

#drop _id column
df = df.drop('_id',axis=1)
df

NameError: name 'df_read' is not defined

# Data Cleaning

### wash macs with unusual length

In [4]:
df = df[df.m.apply(lambda x : True if len(x) == 6 else False)]
df    

,m,a,r,t
0,"[1, 128, 194, 0, 0, 0]",121,169,2022-07-05 23:57:16
1,"[80, 2, 145, 187, 143, 221]",121,159,2022-07-05 23:57:16
2,"[176, 89, 71, 72, 233, 219]",121,160,2022-07-05 23:57:16
3,"[160, 65, 71, 44, 196, 200]",121,163,2022-07-05 23:57:16
4,"[51, 51, 255, 86, 69, 185]",121,159,2022-07-05 23:57:20
...,...,...,...,...
174556,"[88, 102, 186, 101, 140, 144]",66,176,2022-08-07 23:48:38
174557,"[88, 102, 186, 101, 140, 144]",66,177,2022-08-07 23:51:14
174558,"[1, 0, 94, 127, 255, 250]",66,161,2022-08-07 23:51:14
174559,"[1, 0, 94, 127, 255, 250]",66,162,2022-08-07 23:54:42


### wash macs that contains 0

In [5]:
df = df[df.m.apply(lambda x : False if 0 in x else True)]
df

,m,a,r,t
1,"[80, 2, 145, 187, 143, 221]",121,159,2022-07-05 23:57:16
2,"[176, 89, 71, 72, 233, 219]",121,160,2022-07-05 23:57:16
3,"[160, 65, 71, 44, 196, 200]",121,163,2022-07-05 23:57:16
4,"[51, 51, 255, 86, 69, 185]",121,159,2022-07-05 23:57:20
5,"[80, 2, 145, 187, 143, 221]",121,164,2022-07-05 23:57:20
...,...,...,...,...
174552,"[88, 102, 186, 101, 140, 144]",66,177,2022-08-07 23:44:30
174553,"[88, 102, 186, 101, 140, 144]",66,177,2022-08-07 23:46:54
174556,"[88, 102, 186, 101, 140, 144]",66,176,2022-08-07 23:48:38
174557,"[88, 102, 186, 101, 140, 144]",66,177,2022-08-07 23:51:14


### wash fake macs

#### 清除多播mac地址和广播mac地址

**单播地址（unicast address**）表示单一设备、节点；

**多播地址或者组播地址（multicast address 、 group address）** 表示一组设备、节点；

**广播地址（broadcast address）** 是组播的特例，表示所有地址，全用F表示：FF:FF:FF:FF:FF:FF。


如MAC地址：54-BF-64-2B-09-50，第1个字节是54，第2个字节是BF …，而十六进制54写成二进制为：01010100，红色位标识代表的组播地址与单播地址区分位，单播地址红色位一定为0，即MAC地址一的第一个字节一定是2的倍数.

In [6]:
df = df[df.m.apply(lambda x :True if x[0]%2 == 0 else False)]
df

,m,a,r,t
1,"[80, 2, 145, 187, 143, 221]",121,159,2022-07-05 23:57:16
2,"[176, 89, 71, 72, 233, 219]",121,160,2022-07-05 23:57:16
3,"[160, 65, 71, 44, 196, 200]",121,163,2022-07-05 23:57:16
5,"[80, 2, 145, 187, 143, 221]",121,164,2022-07-05 23:57:20
7,"[96, 238, 92, 49, 127, 3]",121,183,2022-07-05 23:57:33
...,...,...,...,...
174552,"[88, 102, 186, 101, 140, 144]",66,177,2022-08-07 23:44:30
174553,"[88, 102, 186, 101, 140, 144]",66,177,2022-08-07 23:46:54
174556,"[88, 102, 186, 101, 140, 144]",66,176,2022-08-07 23:48:38
174557,"[88, 102, 186, 101, 140, 144]",66,177,2022-08-07 23:51:14


根据IEEE 802 MAC地址的定义，若MAC地址第一个字节的从低位开始的第二个bit为1，则表示该MAC地址类型为本地管理类型，本地管理类型MAC地址不保证是全局唯一的。随机MAC一般选用本地管理类型的MAC地址。

In [7]:
def IsFakeMac(x):
    x = format(x[0],'04b')
    n = len(x) - 2
    if x[n] == '1':
        return True
    else:
        return False
    
df = df[~df.m.apply(IsFakeMac)]
df

,m,a,r,t
1,"[80, 2, 145, 187, 143, 221]",121,159,2022-07-05 23:57:16
2,"[176, 89, 71, 72, 233, 219]",121,160,2022-07-05 23:57:16
3,"[160, 65, 71, 44, 196, 200]",121,163,2022-07-05 23:57:16
5,"[80, 2, 145, 187, 143, 221]",121,164,2022-07-05 23:57:20
7,"[96, 238, 92, 49, 127, 3]",121,183,2022-07-05 23:57:33
...,...,...,...,...
174552,"[88, 102, 186, 101, 140, 144]",66,177,2022-08-07 23:44:30
174553,"[88, 102, 186, 101, 140, 144]",66,177,2022-08-07 23:46:54
174556,"[88, 102, 186, 101, 140, 144]",66,176,2022-08-07 23:48:38
174557,"[88, 102, 186, 101, 140, 144]",66,177,2022-08-07 23:51:14


### Convert Mac to String

In [8]:
def convertMac2Str(x):
    list_s = map(str,x)
    s = ','.join(list_s)
    return s

df['m'] = df['m'].apply(convertMac2Str)
df.head()

,m,a,r,t
1,"80,2,145,187,143,221",121,159,2022-07-05 23:57:16
2,"176,89,71,72,233,219",121,160,2022-07-05 23:57:16
3,"160,65,71,44,196,200",121,163,2022-07-05 23:57:16
5,"80,2,145,187,143,221",121,164,2022-07-05 23:57:20
7,"96,238,92,49,127,3",121,183,2022-07-05 23:57:33


In [9]:
df = df.reindex(columns = ['a','r','t','m'])
df.head()

,a,r,t,m
1,121,159,2022-07-05 23:57:16,"80,2,145,187,143,221"
2,121,160,2022-07-05 23:57:16,"176,89,71,72,233,219"
3,121,163,2022-07-05 23:57:16,"160,65,71,44,196,200"
5,121,164,2022-07-05 23:57:20,"80,2,145,187,143,221"
7,121,183,2022-07-05 23:57:33,"96,238,92,49,127,3"


### wash macs that only appeared once

In [10]:
df = df[df.duplicated("m",keep = False)]
df

,a,r,t,m
1,121,159,2022-07-05 23:57:16,"80,2,145,187,143,221"
2,121,160,2022-07-05 23:57:16,"176,89,71,72,233,219"
3,121,163,2022-07-05 23:57:16,"160,65,71,44,196,200"
5,121,164,2022-07-05 23:57:20,"80,2,145,187,143,221"
7,121,183,2022-07-05 23:57:33,"96,238,92,49,127,3"
...,...,...,...,...
174552,66,177,2022-08-07 23:44:30,"88,102,186,101,140,144"
174553,66,177,2022-08-07 23:46:54,"88,102,186,101,140,144"
174556,66,176,2022-08-07 23:48:38,"88,102,186,101,140,144"
174557,66,177,2022-08-07 23:51:14,"88,102,186,101,140,144"


### wash macs that only appeared in one place

In [11]:
list_count = df.groupby(['m']).a.value_counts()
dd = list_count.to_frame().rename(columns={'a':'A'}).reset_index()
mac_Once = dd[~dd.duplicated('m', keep=False)].m.reset_index().drop('index',axis=1)
mac_Once

,m
0,"100,145,159,224,55,191"
1,"100,145,159,48,148,53"
2,"100,145,162,146,189,25"
3,"100,145,163,172,115,82"
4,"100,145,163,48,148,53"
...,...
8196,"96,98,173,154,151,204"
8197,"96,98,173,86,34,94"
8198,"96,98,175,32,50,51"
8199,"96,98,175,86,34,94"


In [12]:
#删除mac_Once对应的mac地址
df = df.loc[~df.m.isin(mac_Once.m)]
df

,a,r,t,m
12605,22,171,2022-07-05 23:49:36,"48,74,38,155,214,98"
12606,22,163,2022-07-05 23:49:36,"252,107,240,89,142,53"
12607,22,170,2022-07-05 23:49:36,"164,125,159,153,152,106"
12615,22,166,2022-07-05 23:49:47,"252,107,240,89,142,53"
12616,22,170,2022-07-05 23:49:47,"164,125,159,153,152,106"
...,...,...,...,...
174552,66,177,2022-08-07 23:44:30,"88,102,186,101,140,144"
174553,66,177,2022-08-07 23:46:54,"88,102,186,101,140,144"
174556,66,176,2022-08-07 23:48:38,"88,102,186,101,140,144"
174557,66,177,2022-08-07 23:51:14,"88,102,186,101,140,144"


### Convert RSSI

In [13]:
df.r = df.loc[:,'r'].apply(lambda x : format(x,'04b'))

def bin2dec(b):
    number = 0
    counter = 0
    for i in b[::-1]: 
        number += int(i)*(2**counter)
        counter += 1

    return number

def Rssi2dec(r):
    n = bin2dec(r[1:8])
    if r[0] == '0':
        return n
    else:
        return -n
    
df.r = df.r.apply(Rssi2dec)
df

,a,r,t,m
12605,22,-43,2022-07-05 23:49:36,"48,74,38,155,214,98"
12606,22,-35,2022-07-05 23:49:36,"252,107,240,89,142,53"
12607,22,-42,2022-07-05 23:49:36,"164,125,159,153,152,106"
12615,22,-38,2022-07-05 23:49:47,"252,107,240,89,142,53"
12616,22,-42,2022-07-05 23:49:47,"164,125,159,153,152,106"
...,...,...,...,...
174552,66,-49,2022-08-07 23:44:30,"88,102,186,101,140,144"
174553,66,-49,2022-08-07 23:46:54,"88,102,186,101,140,144"
174556,66,-48,2022-08-07 23:48:38,"88,102,186,101,140,144"
174557,66,-49,2022-08-07 23:51:14,"88,102,186,101,140,144"


In [15]:
df.to_csv(os.getcwd()+'\\wifi_track_Data\\dacang\\track_data\\dacang_track_data_1_1214.csv',index = False)